In [ ]:
key = ''

In [ ]:
!pip3 install -U langchain openai pandas
import langchain
import openai
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.callbacks import get_openai_callback 

In [ ]:
import os
base_dir = "/kaggle/input/melee-characters/"

def parse_cost(cost):
    return float(str(cost).split("\n")[-1].split(" ")[-1].replace("$",""))

def fetch_data(character):
    file_path = base_dir + f"{character}.txt"
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def get_chunks(text_str, num_chunks):
    chunk_size = len(text_str) // num_chunks
    chunks = [text_str[i:i + chunk_size] for i in range(0, len(text_str), chunk_size)]
    return chunks

def get_character_data(character, num_chunks):
    text = fetch_data(character)
    chunks = get_chunks(text, num_chunks)
    return chunks[:-1]


In [ ]:
characters = os.listdir(base_dir)
characters = [x.replace('.txt','') for x in characters]

In [ ]:
model_gpt35 = "gpt-3.5-turbo-16k"
chatgpt = ChatOpenAI(temperature = 0.7, model = model_gpt35, openai_api_key = key)

In [ ]:
import numpy as np

# Less varied FAQ's if smaller chunk size as ChatGPT has a more constrained possibility
# of FAQs to make. Try to force smaller while still showing randomness

## Normally: 
# mean = 3
# std = 3.5
# min = 2
# max = 50

## For fox:
# mean = 45
# std_dev = 3
# min_val = 35

def sample_chunk_size(num_samples, mean, std_dev, min_val, max_val):
    mean = mean
    std_deviation = std_dev  # Adjust this value based on your desired standard deviation
    min_value = min_val
    max_value = max_val
    num_samples = num_samples  # Adjust the number of samples as needed

    samples = np.random.normal(mean, std_deviation, num_samples)
    samples = np.clip(samples, min_value, max_value)
    samples = [int(x) for x in samples]

    return samples

In [ ]:
starting_prompt="""
You are a helpful assistant that takes user-written guides on playable characters from Super Smash Brothers Melee for the Nintendo Gamecube and creates questions and answers based on the guide.
As you are producing question and answer pairs for competitive players, please try to ensure the questions and answers are not overly simplistic."""

In [ ]:
def enrich_prompt(starting_prompt):
    starting_prompt += "\n"
    technical_decider = np.random.choice([0,1])
    if technical_decider == 1:
        print('Question made more technical')
        starting_prompt += "Questions should be as technical as possible.\n"
        
    length_decider = np.random.choice([0,1,2])
    if length_decider == 1:
        print('Answer lengthy')
        starting_prompt += "Answers should be lengthy.\n"
    if length_decider == 0:
        print('Answer concise')
        starting_prompt += "Answers should be concise.\n"
    else:
        starting_prompt += ""
        
    starting_prompt+="""
Do not mention the guide itself in any way. Split the question answer pairs by two newlines.

#Guide#: 
{guide}

#Generated questions and answers#:
    """
    
    return starting_prompt


In [ ]:
print(enrich_prompt(starting_prompt))

In [ ]:
from openai import InvalidRequestError
from openai.error import RateLimitError
import logging
import io
from langchain.llms.base import logger

In [ ]:
results = []

In [ ]:
costs = 0
its = 20

for character in characters:
    print('Character: ',character)
    if character == 'fox': ## By far the most data, so smaller. chunks will max context length
        print('Doing fox. Need to use larger chunk size with mean ', 30)
        chunk_sizes = sample_chunk_size(num_samples = its, mean = 30, std_dev = 3, min_val = 20, max_val = 50)
    else:
        print('Using chunk size with mean ', 12)
        chunk_sizes = sample_chunk_size(num_samples = its, mean = 12, std_dev = 5, min_val = 2, max_val = 50)
    for i in chunk_sizes:
        print('New it')
        num_chunks = i
        print('num chunks:',num_chunks)
        text = get_character_data(character, num_chunks)
        for chunk in text:
            try:
                log_capture_buffer = io.StringIO()
                with get_openai_callback() as cost:
                    template = ChatPromptTemplate.from_messages([
                        ("system", enrich_prompt(starting_prompt)),
                    ])
                    response = chatgpt(template.format_messages(guide = chunk))
                    for qa in response.content.split("\n\n"):
                        results.append({"character":character, "qa":qa})
                    costs += parse_cost(cost)
            except InvalidRequestError as invalidrequest:
                print(invalidrequest)
            except Exception as err:
                print(err)
            finally:
                log_handler = logging.StreamHandler(log_capture_buffer)
                logger.addHandler(log_handler)
                log_handler.flush()
                captured_logs = log_capture_buffer.getvalue()
                if "RateLimitError" in captured_logs:
                    print('Sleeping for rate limit error')
                    time.sleep(61)
                    log_handler = logging.StreamHandler(log_capture_buffer)

                # Remove the log handler to prevent duplicate logs
                logger.removeHandler(log_handler)

    print(f'Billed so far: {costs}')

In [ ]:
import pandas as pd
df = pd.DataFrame(results)

In [ ]:
df.to_csv(f'random_chunk_{its}its.csv')